In [6]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.regression.rolling import RollingOLS
import random

# Read the Excel files
index_data = pd.read_excel('DATA Refinitiv.xlsx')
macro_data = pd.read_excel('Stationarity Test Macro Data.xlsx')

# Define the list of indices and macroeconomic variables
indices = ['DAX ', 'Stoxx', 'S&P 500', 'Dow Jones Industria', 'Bovespa Index', 'Russell 2000', 'FTSE 100', 'Hang Seng Index', 'NASDAQ-100', 'CAC 40']
macro_variables = ['CPI', 'EMP_US', 'Import in P. US', 'UNEMP_US', 'Retail', 'INT. TRADE', 'INDST_Prod', 'MONEY SUPPLY', 'TREASURY', 'PROD_Energy']

# Create a table to store the results
results_table = pd.DataFrame(columns=['Index', 'Variables', 'R-squared', 'Chow Test', 'Rolling Betas Stability'])

# Test multivariate OLS models for each index
for index in indices:
    best_r_squared = 0
    best_variables = None
    beta_stability_list = []  # List to store beta stabilities

    for i in range(5):
        # Select a random combination of macroeconomic variables
        selected_variables = random.sample(macro_variables, 5)  # 5 Variables 

        # Combine the index and macroeconomic data
        data = pd.concat([index_data[index], macro_data[selected_variables]], axis=1)

        # Drop rows with missing values
        data = data.dropna()

        # Define the dependent variable (index)
        y = data[index]

        # Define the independent variables (macro variables)
        X = data[selected_variables]

        # Add a constant term for the intercept
        X = sm.add_constant(X)

        # Fit the OLS model
        model = sm.OLS(y, X).fit()

        # Check if the current model has higher R-squared than the best model
        if model.rsquared > best_r_squared:
            best_r_squared = model.rsquared
            best_variables = selected_variables

    # Split the data into two subsets
    n = len(data)
    split_point = n // 2

    # Fit the model for the first subset
    X1 = sm.add_constant(X[:split_point])
    model_1 = sm.OLS(y[:split_point], X1, missing='drop').fit()

    # Fit the model for the second subset
    X2 = sm.add_constant(X[split_point:])
    model_2 = sm.OLS(y[split_point:], X2, missing='drop').fit()

    # Calculate the sum of squared residuals for each subset
    RSS_1 = sum(model_1.resid ** 2)
    RSS_2 = sum(model_2.resid ** 2)

    # Calculate the Chow test statistic
    chow_test = RSS_1 / RSS_2

    # Perform the Rolling Window Regression
    window_size = 60  # 60 are 5 years, which makes sense with monthly data
    rolling_beta = RollingOLS(y, X, window=window_size).fit()

    # Check the stability of the rolling betas
    rolling_betas = rolling_beta.params
    beta_stability = np.std(rolling_betas, axis=0)  # Standard deviation of rolling betas
    beta_stability_list.append(beta_stability)  # Append the standard deviation to the list

    # Calculate the mean of the rolling betas' standard deviation
    beta_stability_mean = np.mean(beta_stability_list)

    # Store the results in the table
    results_table = results_table.append({
        'Index': index,
        'Variables': ', '.join(best_variables),
        'R-squared': best_r_squared,
        'Chow Test': chow_test,
        'Rolling Betas Stability': beta_stability_mean
    }, ignore_index=True)

# Sort the results table by the mean standard deviation in descending order
results_table = results_table.sort_values(by='Rolling Betas Stability', ascending=True)

# Print the sorted results table
print(results_table)


/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/2376977863.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/2376977863.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/2376977863.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/2376977863.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_tab

                 Index                                          Variables  \
1                Stoxx    INDST_Prod, MONEY SUPPLY, UNEMP_US, EMP_US, CPI   
5         Russell 2000  PROD_Energy, Import in P. US, CPI, UNEMP_US, M...   
7      Hang Seng Index  MONEY SUPPLY, UNEMP_US, Import in P. US, TREAS...   
6             FTSE 100  TREASURY, CPI, INDST_Prod, Import in P. US, UN...   
2              S&P 500  CPI, PROD_Energy, Import in P. US, UNEMP_US, M...   
3  Dow Jones Industria  Import in P. US, CPI, EMP_US, MONEY SUPPLY, Re...   
9               CAC 40  MONEY SUPPLY, TREASURY, INDST_Prod, Retail, Im...   
0                 DAX       MONEY SUPPLY, CPI, UNEMP_US, EMP_US, TREASURY   
8           NASDAQ-100  EMP_US, TREASURY, MONEY SUPPLY, Import in P. U...   
4        Bovespa Index  INDST_Prod, Import in P. US, UNEMP_US, Retail,...   

  R-squared  Chow Test Rolling Betas Stability  
1  0.505007   1.882153               36.019906  
5  0.767848  14.570002              130.035062  
7  0.

/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/2376977863.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/2376977863.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/2376977863.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({


In [7]:
results_table

,Index,Variables,R-squared,Chow Test,Rolling Betas Stability
1,Stoxx,"INDST_Prod, MONEY SUPPLY, UNEMP_US, EMP_US, CPI",0.505007,1.882153,36.019906
5,Russell 2000,"PROD_Energy, Import in P. US, CPI, UNEMP_US, M...",0.767848,14.570002,130.035062
7,Hang Seng Index,"MONEY SUPPLY, UNEMP_US, Import in P. US, TREAS...",0.270514,1.681692,178.368901
6,FTSE 100,"TREASURY, CPI, INDST_Prod, Import in P. US, UN...",0.496514,2.66971,431.654733
2,S&P 500,"CPI, PROD_Energy, Import in P. US, UNEMP_US, M...",0.880586,21.198921,453.010831
3,Dow Jones Industria,"Import in P. US, CPI, EMP_US, MONEY SUPPLY, Re...",0.802586,24.062305,594.033008
9,CAC 40,"MONEY SUPPLY, TREASURY, INDST_Prod, Retail, Im...",0.619632,2.932133,618.507598
0,DAX,"MONEY SUPPLY, CPI, UNEMP_US, EMP_US, TREASURY",0.503557,4.741972,1182.64491
8,NASDAQ-100,"EMP_US, TREASURY, MONEY SUPPLY, Import in P. U...",0.924929,29.230003,1654.409253
4,Bovespa Index,"INDST_Prod, Import in P. US, UNEMP_US, Retail,...",0.349476,0.950593,3801.49012


## Results aavaluated by Chrow test

In [9]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.regression.rolling import RollingOLS
import random

# Read the Excel files
index_data = pd.read_excel('DATA Refinitiv.xlsx')
macro_data = pd.read_excel('Stationarity Test Macro Data.xlsx')

# Define the list of indices and macroeconomic variables
indices = ['DAX ', 'Stoxx', 'S&P 500', 'Dow Jones Industria', 'Bovespa Index', 'Russell 2000', 'FTSE 100', 'Hang Seng Index', 'NASDAQ-100', 'CAC 40']
macro_variables = ['CPI', 'EMP_US', 'Import in P. US', 'UNEMP_US', 'Retail', 'INT. TRADE', 'INDST_Prod', 'MONEY SUPPLY', 'TREASURY', 'PROD_Energy']

# Create a table to store the results
results_table = pd.DataFrame(columns=['Index', 'Variables', 'R-squared', 'Chow Test', 'Rolling Betas Stability'])

# Test multivariate OLS models for each index
for index in indices:
    best_r_squared = 0
    best_variables = None
    beta_stability_list = []  # List to store beta stabilities

    for i in range(5):
        # Select a random combination of macroeconomic variables
        selected_variables = random.sample(macro_variables, 5)  # 5 Variables 

        # Combine the index and macroeconomic data
        data = pd.concat([index_data[index], macro_data[selected_variables]], axis=1)

        # Drop rows with missing values
        data = data.dropna()

        # Define the dependent variable (index)
        y = data[index]

        # Define the independent variables (macro variables)
        X = data[selected_variables]

        # Add a constant term for the intercept
        X = sm.add_constant(X)

        # Fit the OLS model
        model = sm.OLS(y, X).fit()

        # Check if the current model has higher R-squared than the best model
        if model.rsquared > best_r_squared:
            best_r_squared = model.rsquared
            best_variables = selected_variables

    # Split the data into two subsets
    n = len(data)
    split_point = n // 2

    # Fit the model for the first subset
    X1 = sm.add_constant(X[:split_point])
    model_1 = sm.OLS(y[:split_point], X1, missing='drop').fit()

    # Fit the model for the second subset
    X2 = sm.add_constant(X[split_point:])
    model_2 = sm.OLS(y[split_point:], X2, missing='drop').fit()

    # Calculate the sum of squared residuals for each subset
    RSS_1 = sum(model_1.resid ** 2)
    RSS_2 = sum(model_2.resid ** 2)

    # Calculate the Chow test statistic
    chow_test = RSS_1 / RSS_2

    # Perform the Rolling Window Regression
    window_size = 60  # 60 are 5 years, which makes sense with monthly data
    rolling_beta = RollingOLS(y, X, window=window_size).fit()

    # Check the stability of the rolling betas
    rolling_betas = rolling_beta.params
    beta_stability = np.std(rolling_betas, axis=0)  # Standard deviation of rolling betas
    beta_stability_list.append(beta_stability)  # Append the standard deviation to the list

    # Calculate the mean of the rolling betas' standard deviation
    beta_stability_mean = np.mean(beta_stability_list)

    # Store the results in the table
    results_table = results_table.append({
        'Index': index,
        'Variables': ', '.join(best_variables),
        'R-squared': best_r_squared,
        'Chow Test': chow_test,
        'Rolling Betas Stability': beta_stability_mean
    }, ignore_index=True)

# Sort the results table by the mean standard deviation in descending order
results_table = results_table.sort_values(by='Chow Test', ascending=True)

# Print the sorted results table
print(results_table)


/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/506545336.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/506545336.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/506545336.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/506545336.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table =

                 Index                                          Variables  \
4        Bovespa Index  CPI, UNEMP_US, PROD_Energy, INDST_Prod, Import...   
6             FTSE 100  EMP_US, Retail, Import in P. US, UNEMP_US, IND...   
3  Dow Jones Industria  TREASURY, INT. TRADE, EMP_US, Import in P. US,...   
1                Stoxx  PROD_Energy, CPI, INDST_Prod, MONEY SUPPLY, Im...   
7      Hang Seng Index  MONEY SUPPLY, Retail, TREASURY, Import in P. U...   
9               CAC 40  INDST_Prod, PROD_Energy, MONEY SUPPLY, Import ...   
0                 DAX   MONEY SUPPLY, UNEMP_US, INDST_Prod, PROD_Energ...   
5         Russell 2000  MONEY SUPPLY, INT. TRADE, TREASURY, CPI, UNEMP_US   
2              S&P 500  MONEY SUPPLY, Import in P. US, TREASURY, INT. ...   
8           NASDAQ-100      UNEMP_US, EMP_US, MONEY SUPPLY, CPI, TREASURY   

  R-squared  Chow Test Rolling Betas Stability  
4  0.277911    1.38389            11333.687272  
6  0.441587   1.496753              1292.04472  
3  0.

/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/506545336.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
/var/folders/x5/r7dhf7_522xc1q4b62kxvbb00000gn/T/ipykernel_16043/506545336.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({


In [10]:
results_table

,Index,Variables,R-squared,Chow Test,Rolling Betas Stability
4,Bovespa Index,"CPI, UNEMP_US, PROD_Energy, INDST_Prod, Import...",0.277911,1.38389,11333.687272
6,FTSE 100,"EMP_US, Retail, Import in P. US, UNEMP_US, IND...",0.441587,1.496753,1292.04472
3,Dow Jones Industria,"TREASURY, INT. TRADE, EMP_US, Import in P. US,...",0.814209,2.228798,278.568232
1,Stoxx,"PROD_Energy, CPI, INDST_Prod, MONEY SUPPLY, Im...",0.475871,2.811938,61.449056
7,Hang Seng Index,"MONEY SUPPLY, Retail, TREASURY, Import in P. U...",0.312581,4.507716,532.124747
9,CAC 40,"INDST_Prod, PROD_Energy, MONEY SUPPLY, Import ...",0.620973,4.601232,441.926046
0,DAX,"MONEY SUPPLY, UNEMP_US, INDST_Prod, PROD_Energ...",0.527491,4.682233,1321.491403
5,Russell 2000,"MONEY SUPPLY, INT. TRADE, TREASURY, CPI, UNEMP_US",0.757791,13.303459,117.09824
2,S&P 500,"MONEY SUPPLY, Import in P. US, TREASURY, INT. ...",0.907452,20.129138,464.976287
8,NASDAQ-100,"UNEMP_US, EMP_US, MONEY SUPPLY, CPI, TREASURY",0.915197,20.858191,866.819037
